In [1]:
# %pip install --upgrade domolibrary

In [22]:
import domolibrary.client.DomoAuth as dmda
import domojupyter as dj


async def generate_domo_auth(account_name : str, 
                             domo_instance : str,
                             debug_prn: bool = False) -> dmda.DomoAuth:
    
    account_properties = dj.get_account_property_keys(account_name)
    if debug_prn:
        print(account_properties)
    
    creds = {prop : dj.get_account_property_value(account_name, prop) for prop in account_properties}

    auth = dmda.DomoTokenAuth(domo_access_token = creds['domoAccessToken'],
                   domo_instance = domo_instance)

    if debug_prn:
        print((await auth.who_am_i()).response)
    
    return auth

auth = await generate_domo_auth( 'sdk_domo-community', 'domo_community', debug_prn = False)

USER_ID = 1893952720

In [38]:
import domolibrary.classes.DomoDataflow as dmdf

async def get_filtered_dataflows(auth: dmda.DomoAuth, user_id : int):
    domo_dataflows = await dmdf.DomoDataflows(auth = auth).get()
    
    return [df for df in domo_dataflows if int(df.owner) == user_id]

filtered_dataflows = await get_filtered_dataflows(auth = auth, user_id = USER_ID)
filtered_dataflows[0]

DomoDataflow(id=131, name='BeastModesGovernance_01_AccumulateHistory', owner=1893952720, description='This JSON describes a Domo ETL dataflow named "BeastModesGovernance_01_AccumulateHistory" with ID 131. The dataflow consists of three actions:\n\n1. LoadFromVault:\n   - Action ID: 40554fc4-428a-4117-8820-b3f4f4927021\n   - Name: "Governance_Beast Modes"\n   - Loads data from a Domo Vault with data source ID b7998b45-ccb5-4183-830e-8c2bdd489fd0\n   - Does not execute flow when updated\n   - Does not truncate text columns or rows\n   - Does not only load new versions\n\n2. ExpressionEvaluator:\n   - Action ID: 05082103-d42e-49ab-a66e-bd31b4c0e3df\n   - Name: "Add Formula"\n   - Depends on the LoadFromVault action\n   - Adds a new column named "report_date" with a fixed value of "2023-03-06"\n\n3. PublishToVault:\n   - Action ID: fe77761b-eb11-4491-b19c-ebb56d009b76\n   - Name: "Governance_BeastMode_historical"\n   - Depends on the ExpressionEvaluator action\n   - Publishes data to a Dom

In [44]:
from enum import Enum

class ETL_BackgroundVariant(Enum):
    NONE = "None"
    DOT = "Dot"
    LINES = "Lines"
    CROSS = "Cross"
    
    
async def process_dataflow(domo_dataflow: dmdf.DomoDataflow, 
                           background_variant : ETL_BackgroundVariant = ETL_BackgroundVariant.NONE
):
    dataflow_definition = (await domo_dataflow.get_definition(return_raw = True)).response
    
    # return dataflow_definition
    
    try:
        dataflow_definition['gui']['canvases']['default']['canvasSettings'].update(
            {"coraserGrid": True,
             "backgroundVariant": background_variant.value}
        )    

        return await domo_dataflow.update_dataflow_definition(dataflow_definition, debug_api = False)
    
    except KeyError as e:
        return domo_dataflow
    
    except dmde.DomoError as e:
        print(e)
        
await process_dataflow(filtered_dataflows[0], background_variant = ETL_BackgroundVariant.CROSS)


DomoDataflow(id=131, name='BeastModesGovernance_01_AccumulateHistory', owner=1893952720, description='This JSON describes a Domo ETL dataflow named "BeastModesGovernance_01_AccumulateHistory" with ID 131. The dataflow consists of three actions:\n\n1. LoadFromVault:\n   - Action ID: 40554fc4-428a-4117-8820-b3f4f4927021\n   - Name: "Governance_Beast Modes"\n   - Loads data from a Domo Vault with data source ID b7998b45-ccb5-4183-830e-8c2bdd489fd0\n   - Does not execute flow when updated\n   - Does not truncate text columns or rows\n   - Does not only load new versions\n\n2. ExpressionEvaluator:\n   - Action ID: 05082103-d42e-49ab-a66e-bd31b4c0e3df\n   - Name: "Add Formula"\n   - Depends on the LoadFromVault action\n   - Adds a new column named "report_date" with a fixed value of "2023-03-06"\n\n3. PublishToVault:\n   - Action ID: fe77761b-eb11-4491-b19c-ebb56d009b76\n   - Name: "Governance_BeastMode_historical"\n   - Depends on the ExpressionEvaluator action\n   - Publishes data to a Dom

In [45]:
import domolibrary.utils.chunk_execution as dmce

async def update_all_dataflows(auth: dmda.DomoAuth, user_id : int ):
    domo_dataflows = await get_filtered_dataflows(auth = auth, user_id = user_id)
    
    return await dmce.gather_with_concurrency(*[process_dataflow(domo_dataflow = df, 
                                                                 background_variant = ETL_BackgroundVariant.DOT) for df in domo_dataflows], n =10)

res = await update_all_dataflows(auth = auth, user_id = USER_ID)